# Data Mining (KEN4113) #
## Lab 3: Classification ##

Konstantin Sandfort
i6255681

In [39]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold

## Step 1: EDA
The EDA will provide better insights of the data and give us a better idea of the variables that can be used for classification.

In [40]:
# Import data
data_train = pd.read_csv('caravan.csv')
data_test = pd.read_csv('caravanTest.csv')
y = data_train['CARAVAN POLICY']

0       0
1       0
2       0
3       0
4       0
       ..
5817    0
5818    0
5819    1
5820    0
5821    0
Name: CARAVAN POLICY, Length: 5822, dtype: int64


In [3]:
print(data_train.columns)

Index(['Customer Subtype', 'Number of houses', 'Avg size household', 'Avg Age',
       'Customer main type', 'Roman catholic', 'Protestant', 'Other religion',
       'No religion', 'Married', 'Living together', 'Other relation',
       'Singles', 'Household without children', 'Household with children',
       'High level education', 'Medium level education',
       'Lower level education', 'High status', 'Entrepreneur', 'Farmer',
       'Middle Management', 'Skilled labourers', 'Unskilled labourers',
       'Social class A', 'Social class B1', 'Social class B2',
       'Social class C', 'Social class D', 'Rented house', 'Home owners',
       '1 car', '2 cars', 'No car', 'National Health Service',
       'Private health insurance', 'Income < 30.000', 'Income 30-45.000',
       'Income 45-75.000', 'Income 75-122.000', 'Income >123.000',
       'Average income', 'Purchasing power class',
       'Contribution private third party insurance',
       'Contribution third party insurance (firms

In [5]:
pd.set_option('display.max_columns', None)
print(data_train.head(10))

   Customer Subtype  Number of houses  Avg size household  Avg Age  \
0                33                 1                   3        2   
1                37                 1                   2        2   
2                37                 1                   2        2   
3                 9                 1                   3        3   
4                40                 1                   4        2   
5                23                 1                   2        1   
6                39                 2                   3        2   
7                33                 1                   2        3   
8                33                 1                   2        4   
9                11                 2                   3        3   

   Customer main type  Roman catholic  Protestant  Other religion  \
0                   8               0           5               1   
1                   8               1           4               1   
2                   8 

In [24]:
# --- Step 1: Data Preprocessing ---
# Step 1.1: One-hot encoding of categorical variables
# Customer subtype
def ohe(df, col_name):
    """
    Applies one-hot-encoding to a column, deletes the original one and replaces it with encoding
    :param df: 
    :param col_name: 
    :return: full ohe df
    """
    cols = pd.get_dummies(df[col_name])
    df_ohe = df.drop(df[col_name])
    return pd.concat([cols, df_ohe], axis=1)

data_train_ohe = ohe(data_train, 'Customer Subtype')
#data_train_ohe = ohe(data_train_ohe, 'Customer main type')
print(data_train_ohe.columns)

Index([                                              1,
                                                     2,
                                                     3,
                                                     4,
                                                     5,
                                                     6,
                                                     7,
                                                     8,
                                                     9,
                                                    10,
       ...
       'Number of private accident insurance policies',
       'Number of family accidents insurance policies',
             'Number of disability insurance policies',
                               'Number of re policies',
                        'Number of surfboard policies',
                             'Number of boat policies',
                          'Number of bicycle policies',
               'Number of property in

In [38]:
# --- Step 2: Feature Selection
# a) Removing variables with low variance

def variance_threshold_keep_col_names(df, thr):
    var_threshold = VarianceThreshold(thr)
    var_threshold.fit_transform(df)
    return df[df.columns[var_threshold.get_support(indices=True)]]

data_train_new = variance_threshold_keep_col_names(data_train, 0.8 * (1 - 0.8))

#print(new_cols)
print(data_train_new.columns)

Index(['Customer Subtype', 'Number of houses', 'Avg size household', 'Avg Age',
       'Customer main type', 'Roman catholic', 'Protestant', 'Other religion',
       'No religion', 'Married', 'Living together', 'Other relation',
       'Singles', 'Household without children', 'Household with children',
       'High level education', 'Medium level education',
       'Lower level education', 'High status', 'Entrepreneur', 'Farmer',
       'Middle Management', 'Skilled labourers', 'Unskilled labourers',
       'Social class A', 'Social class B1', 'Social class B2',
       'Social class C', 'Social class D', 'Rented house', 'Home owners',
       '1 car', '2 cars', 'No car', 'National Health Service',
       'Private health insurance', 'Income < 30.000', 'Income 30-45.000',
       'Income 45-75.000', 'Income 75-122.000', 'Income >123.000',
       'Average income', 'Purchasing power class',
       'Contribution private third party insurance',
       'Contribution third party insurance (agric

In [ ]:
def univariate_feature_selection(df, y, k_f):
    df_new = SelectKBest(f_classif, k=k_f).fit_transform(df, y)
    return df_new